# Assignment 3

Date: 20-12-2020 <br>
Nick Radunovic (s2072724) <br>
Cheyenne Heath (s1647865) <br>

In [268]:
#Imports
import random
import os
random.seed(30) # set random seed for reproducibility

import numpy as np
from itertools import chain
from collections import Counter
import eli5

import nltk
nltk.download('punkt')
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Stand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Preprocessing

First, we define function that are used for preprocessing the data

In [269]:
def parse_data(text, annotation):    
    # parse annotation-file
    bio_tag = []
    with open(annotation, encoding='utf-8') as ann:
        for l in ann:
            l = l.split('\t')
            tag = l[1].split()[0]
            if not tag == 'AnnotatorNotes':
                string = l[2].strip()
                bio_tag.append((tag, string))
                
    # parse text-file and add POS tags
    sents = []
    with open(text, encoding='utf-8') as fp:
        for line in fp:
            line = line.strip()
            line = nltk.word_tokenize(line)
            pos_line = add_pos_tag(line)
            sents.append(pos_line)
    
    # add BIO tags and format data as follows -> (word, pos, biotag)
    sents = add_bio_tag(sents, bio_tag)
    
    return sents


def add_pos_tag(sent):
    return nltk.pos_tag(sent)


def add_bio_tag(sents, bio_tag):
    """ We use 'cadec/original/' as annotation to add BIO-tags """
    msg = []
    for sent in sents:
        bio_sent = []
        remaining = 0
        for i, word in enumerate(sent):
            BIOtag = 'O'
            for j, tag in enumerate(bio_tag):
                target = nltk.word_tokenize(tag[1])
                tag = tag[0]
                count = 0
                
                # changes the biotag to either 'B-' or 'I-' when necessary
                if word[0] == target[0]:
                    for k in range(len(target)):
                        if len(sent) > i+k and sent[i+k][0] == target[k]:
                            count += 1
                            
                    # if target is found, the current word gets a 'B-' tag assigned
                    if count == len(target):
                        definite_tag = tag
                        BIOtag = 'B-' + definite_tag
                        remaining = len(target) - 1
                        break
            
            # changes the biotag to 'I-' when necessary
            if remaining > 0 and BIOtag == 'O':
                BIOtag = 'I-' + definite_tag
                remaining -= 1
          
            bio_sent.append((word[0], word[1], BIOtag))
        msg.append(bio_sent)
    return msg

We preprocess the data:

(1) The data is parsed,
(2) POS tags are added,
(3) BIO tags are added.

In [270]:
DIR_text = 'cadec/text/'
DIR_annotation = 'cadec/original/'

data = []
for f in os.listdir(DIR_text):
    text = DIR_text + f
    ann = DIR_annotation + f[:-4] + '.ann'
    m = parse_data(text, ann)
    data.append(m)

# shows first patient post
data[0]

[[('I', 'PRP', 'O'),
  ('feel', 'VBP', 'O'),
  ('a', 'DT', 'O'),
  ('bit', 'NN', 'B-ADR'),
  ('drowsy', 'JJ', 'I-ADR'),
  ('&', 'CC', 'O'),
  ('have', 'VBP', 'O'),
  ('a', 'DT', 'O'),
  ('little', 'JJ', 'B-ADR'),
  ('blurred', 'JJ', 'I-ADR'),
  ('vision', 'NN', 'I-ADR'),
  (',', ',', 'O'),
  ('so', 'RB', 'O'),
  ('far', 'RB', 'O'),
  ('no', 'DT', 'O'),
  ('gastric', 'JJ', 'B-ADR'),
  ('problems', 'NNS', 'I-ADR'),
  ('.', '.', 'O')],
 [('I', 'PRP', 'O'),
  ("'ve", 'VBP', 'O'),
  ('been', 'VBN', 'O'),
  ('on', 'IN', 'O'),
  ('Arthrotec', 'NNP', 'B-Drug'),
  ('50', 'CD', 'O'),
  ('for', 'IN', 'O'),
  ('over', 'IN', 'O'),
  ('10', 'CD', 'O'),
  ('years', 'NNS', 'O'),
  ('on', 'IN', 'O'),
  ('and', 'CC', 'O'),
  ('off', 'IN', 'O'),
  (',', ',', 'O'),
  ('only', 'RB', 'O'),
  ('taking', 'VBG', 'O'),
  ('it', 'PRP', 'O'),
  ('when', 'WRB', 'O'),
  ('I', 'PRP', 'O'),
  ('needed', 'VBD', 'O'),
  ('it', 'PRP', 'O'),
  ('.', '.', 'O')],
 [('Due', 'JJ', 'O'),
  ('to', 'TO', 'O'),
  ('my', 'PRP$', 

## Training